In [ ]:
import os
import subprocess
import pandas as pd

In [ ]:
tr2_script_path = "/tr2-delimitation-python3/run_tr2.py"
soda_script_path = "/SODA/run_delimitation.py"
genetrees_path = "../data/phylo/genetrees_rerooted.nwk"
species_guide_tree_path = "../data/phylo/guide_species_tree_rerooted_expanded.nwk"
tr2_output_path = "../data/phylo/tr2_delimitation"
soda_output_path = "../data/phylo/soda_delimitation.txt"
phylo_dir = "../data/phylo"
astral_path = "/SODA/Astral"

This part runs [tr2-delimitation](https://github.com/tfujisawa/tr2-delimitation-git) (Fujisawa et al. 2016)

In [ ]:
subprocess.run([
    tr2_script_path,
    "-t", genetrees_path,
    "-g", species_guide_tree_path,
    "-o", tr2_output_path,
])

In [ ]:
tr2_delim = pd.read_table(f"{tr2_output_path}.table.txt")
pops = pd.read_table("../data/samples/populations.txt", header=None, names=["sample", "population"])
tr2_delim = tr2_delim.merge(pops, on="sample")
tr2_delim

In [ ]:
for group in tr2_delim.groupby("population"):
    print(f"{group[0]}: {group[1]['species'].tolist()}")

In [ ]:
tr2_delim.to_csv("../data/phylo/tr2_delim.csv", index=False)

This part runs [SODA](https://github.com/maryamrabiee/SODA) (Rabiee and Mirarab, 2020)

In [ ]:
if not os.path.exists("Astral"):
    os.symlink(astral_path, "Astral")
subprocess.run([
    "python3", soda_script_path,
    "-i", genetrees_path,
    "-t", species_guide_tree_path,
    "-o", soda_output_path,
    "-d", phylo_dir,
    "--rooted",
], check=True)
os.remove("Astral")

In [ ]:
soda_delim = pd.read_table(soda_output_path, header=None, names=["sample", "species"])
soda_delim = soda_delim.merge(pops, on="sample")
soda_delim

In [ ]:
for group in soda_delim.groupby("population"):
    print(f"{group[0]}: {group[1]['species'].tolist()}")